<p><img src="https://files.cdn.thinkific.com/file_uploads/133292/images/a9e/c1a/bc3/P02.jpg"></p>

`MCSDSC02P01V3______`

`MCSDSC02P01V2______`

`MCSDSELIMC02P01V1010621`

<h3><font color="#89DD00">Комментарий ассистента</font></h3>

Как обычно, заголовки (комментариев) будут трех цветов. <font color="89DD00">Зеленый</font> — все правильно (и если комментария нет, то это также значит, что все правильно). <font color="FFB800">Желтый</font> — можно доработать. <font color="FF0F00">Красный</font> — нужно исправить.

## Постановка задачи

Изучить данные предоставляемые приложением Bez Ocheredi на территории дизайн-завода Флакон и Хлебозавода №9 и ответить на вопросы, поставленные менеджером проекта:
- На какие кафе приходится больше всего заказов? А на какие дни?
- Чаще ли приложением пользуются именно в обеденное время?
- Насколько большими компаниями  (или наоборот поодиночке) приходят в кафе?

## Импорт данных и библиотек

In [1]:
import pandas as pd

Прочитаем файл `bez_ocheredi.csv` и сохраним его в переменную `bo_data`.

In [2]:
bo_data = pd.read_csv('https://raw.githubusercontent.com/moscoding/mcs_ds_02/main/data/bez_ocheredi.csv?token=AKFSI2DIX6OILVJCMZSYXA3BOJJBS')

Получим первые 10 строк таблицы.

In [3]:
bo_data.head(10)

,userID,ресторан,weekday,ordered,Столик для
0,b912c3c6a6ad,Iskra Bar,Monday,10:00,1
1,2dac0150717d,Iskra Bar,Monday,13:30,2
2,ba83aefab5dc,Iskra Bar,Tuesday,12:30,3
3,02996f1dd2ea,Iskra Bar,Wednesday,12:00,2
4,7c90be56d2ab,Iskra Bar,Thursday,14:30,3
5,51e0962387f7,Там Ем,Friday,12:30,2
6,c4f260a2b48a,NaN,Saturday,15:00,6
7,6b71309d6a8a,Крафтейнер,Monday,10:30,1
8,c5b969cb63a2,Бар Сибирь,Wednesday,15:30,2
9,b1570962e68c,Бар Сибирь,Wednesday,12:30,3


Получим общую информацию о данных из таблицы.

In [4]:
bo_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1024 entries, 0 to 1023
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   userID        1024 non-null   object
 1    ресторан     959 non-null    object
 2    weekday      1024 non-null   object
 3    ordered      1024 non-null   object
 4    Столик для   1024 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 40.1+ KB


In [5]:
bo_data.shape

(1024, 5)

In [6]:
bo_data.isnull().sum() / bo_data.shape[0]

userID          0.000000
 ресторан       0.063477
 weekday        0.000000
 ordered        0.000000
 Столик для     0.000000
dtype: float64

Столбцы в `bo_data` содержат следующую информацию:

* userID - уникальный ID пользователя, в столбце нет пропущенной информации, типом данных является object.
* ресторан - название ресторанов, в данном столбце отсутствует примерно 6% информации, тип данных - object.
* weekday - показывает день недели, в столбце нет пропущенной информации, типом данных является object.
* ordered - показывает на какое время сделан заказ, в столбце нет пропущенной информации, типом данных является object.
* Столик для - содержит количество людей, для которых заказан столик, в столбце нет пропущенной информации, типом данных является int64.

### Выводы

Настоящая таблица предоставляет данные о том, где, когда, и кем был сделан заказ. Информации достаточно, чтобы после процесса предобработки данных ответить на вопросы поставленные менеджером проекта. На данный момент мы видим, что присутствуют все данные, кроме некоторых пропущенных значений в столбце "ресторан". Данная проблема устранится в процессе предобработки данных.

<h3><font color="#89DD00">Комментарий ассистента</font></h3>

Кажется, что здесь вполне хватит `info()`. Он показывает и размерность таблицы и количество пропусков. Здесь нужно было указать на структуру данных, а также наличие пропусков. Все отлично.

## Предобработка данных

Получим названия столбцов.

In [7]:
bo_data.columns

Index(['userID ', ' ресторан ', ' weekday', ' ordered ', ' Столик для '], dtype='object')

Обращаем внимание, что помимо названий на разном языке, в них есть пробелы, которые затрудняют доступ к данным. Переименуем столбцы и проверим результат.

In [8]:
bo_data.rename(columns={'userID ': 'userID', ' ресторан ': 'restaurant', ' weekday': 'weekday', ' ordered ': 'ordered', ' Столик для ': 'table_for'}, inplace=True)

In [9]:
bo_data.columns

Index(['userID', 'restaurant', 'weekday', 'ordered', 'table_for'], dtype='object')

Еще раз проверим данные на наличие пропусков, просуммировав их.

In [10]:
bo_data.isnull().sum()

userID         0
restaurant    65
weekday        0
ordered        0
table_for      0
dtype: int64

Менеджер проекта предупреждал, что некоторые заказы заводились на тестовую карточку кафе без названия, поэтому есть смысл заполнить такой пропуск словом admin.

In [11]:
bo_data.restaurant.fillna('admin', inplace=True)

Проверяем, что пропусков не осталось.

In [12]:
bo_data.isnull().any()

userID        False
restaurant    False
weekday       False
ordered       False
table_for     False
dtype: bool

Установим количество явных дубликатов. Если найдутся, то удалим и снова проверим, что их не осталось.

In [13]:
bo_data.duplicated().sum()

24

In [14]:
bo_data.drop_duplicates(inplace=True)

In [15]:
bo_data.reset_index(drop = True, inplace=True)

In [16]:
bo_data.duplicated().sum()

0

<h3><font color="#89DD00">Комментарий ассистента</font></h3>

Здесь тоже все отлично, дубликаты верно удалены.

Скорее всего, такие полные дубликаты говорят о сбое при выгрузке данных. Необходимо сообщить об этом.

Еще одна проблема, о которой предупреждал менеджер проекта — дубликаты карточки Iskra Bar с неправильными названием бара - "Искра" и "Искр". На них заводились реальные заказы, поэтому необходимо вычислить их и переименовать на правильное название — Iskra Bar.

Объявим функцию `replace_restaurant()`, которая заменяет неправильное название, а потом проверяет успешность выполнения замены подсчетом (если замена прошла успешно — результат подсчета должен быть 0).

In [17]:
def replace_restaurant(df,word, correct_name, column):
    bo_data.replace(word, correct_name, inplace= True)
    if bo_data[column].eq(word).sum() != True:
        print('Done!')

Передаем функции названия 'Искра' и 'Искр'.

In [18]:
replace_restaurant(bo_data, 'Искра', 'Iskra Bar', 'restaurant')

Done!


In [19]:
replace_restaurant(bo_data, 'Искр', 'Iskra Bar', 'restaurant')

Done!


<h3><font color="#89DD00">Комментарий ассистента</font></h3>

Интересный способ проверки. Возможно, стоит тогда убрать про 0 в описании шага и заменить на свое.

In [1]:
bo_data.restaurant.value_counts()

NameError: name 'bo_data' is not defined

In [21]:
bo_data.shape

(1000, 5)

### Выводы

В процессе предобработки данных мы обнаружили, что:
- таблица содержит дубликаты (все дубликаты удалены);
- у некоторых ресторанов отсутствовало название (все пробелы заменены на admin);
- бар Iskra bar был записан разными способами (название бара было унифицировано).

## Распределение заказов в течение недели

Считаем, на какие кафе приходится больше всего заказов.

In [22]:
bo_data.restaurant.value_counts()

Iskra Bar     287
Там Ем        200
Бар Сибирь    159
Северия       107
Крафтейнер     78
Коворкафе      66
Азия           62
admin          41
Name: restaurant, dtype: int64

Группируем по `weekday` и считаем, на какие дни приходится больше всего заказов.

In [23]:
bo_data.groupby('weekday').restaurant.value_counts()

weekday    restaurant
Friday     Iskra Bar     29
           Там Ем        24
           Бар Сибирь    16
           Северия       15
           Азия          11
           Крафтейнер     9
           Коворкафе      5
           admin          1
Monday     Iskra Bar     48
           Там Ем        25
           Бар Сибирь    20
           Северия       15
           Крафтейнер    11
           admin         10
           Азия           6
           Коворкафе      6
Saturday   Iskra Bar     16
           Там Ем        11
           Северия        7
           Бар Сибирь     6
           Крафтейнер     5
           admin          3
           Азия           2
           Коворкафе      2
Sunday     Там Ем        11
           Iskra Bar      7
           Бар Сибирь     4
           Азия           1
           Северия        1
Thursday   Iskra Bar     63
           Бар Сибирь    30
           Там Ем        28
           Северия       20
           Коворкафе     12
           Крафтейнер    1

Объявляем функцию, которая возвращает количество заказов для ресторана в определенный день.

In [24]:
def day_order_count(restaurant,weekday):
    df = bo_data[(bo_data.restaurant == restaurant) & (bo_data.weekday == weekday)]
    return print(f'The number of orders for {restaurant} on {weekday} is {df.shape[0]}.')

Передадим ей заведения Iskra Bar и Там Ем — два флагманских места, по которым просили результаты прежде всего — и понедельник, среду и пятницу в качестве дня недели.

In [25]:
day_order_count('Iskra Bar', 'Monday')

The number of orders for Iskra Bar on Monday is 48.


In [26]:
day_order_count('Iskra Bar', 'Wednesday')

The number of orders for Iskra Bar on Wednesday is 53.


In [27]:
day_order_count('Iskra Bar', 'Friday')

The number of orders for Iskra Bar on Friday is 29.


In [28]:
day_order_count('Там Ем', 'Monday')

The number of orders for Там Ем on Monday is 25.


In [29]:
day_order_count('Там Ем', 'Wednesday')

The number of orders for Там Ем on Wednesday is 52.


In [30]:
day_order_count('Там Ем', 'Friday')

The number of orders for Там Ем on Friday is 24.


Сведем результат в таблицу.

In [31]:
data_pubs = [['Iskra Bar', 48, 53, 29], ['Там Ем', 25, 52, 24]]
columns_pubs = ['restaurant', 'Monday', 'Wednesday', 'Friday']
table_pubs = pd.DataFrame(data=data_pubs, columns=columns_pubs)
table_pubs

,restaurant,Monday,Wednesday,Friday
0,Iskra Bar,48,53,29
1,Там Ем,25,52,24


### Выводы

Изучив распределение заказов в течение недели, мы обнаружили, что тремя самыми популярными ресторанами являются: Iskra Bar, "Там Ем" и "Бар Сибирь". Больше всего заказов в рестораны поступает по будням, и меньше всего на выходных.

<h3><font color="#89DD00">Комментарий ассистента</font></h3>

Да, здесь все правильно. Пик приходится на середину недели.

## Распределение заказов в течение дня

Сравним количество заказов, сделанных в обеденное и вечернее время — с 12 до 16 и с 16 до 22. Для этого объявим функцию, которая принимает данные и 2 аргумента времени.

In [32]:
def ordered_time(time_1, time_2):
    return bo_data.ordered[(bo_data['ordered'] >= time_1) & (bo_data['ordered'] < time_2)].count()

Передадим промежуток с 12:00 и 16:00

In [33]:
ordered_time('12:00','16:00')

543

И с 16:00 до 22:00

In [34]:
ordered_time('16:00','22:00')

338

### Выводы

* Расчитав распределение заказов в течение дня, мы обнаружили, что больше всего заказов приходится на обеденное время, а именно в промежуток между 12:00 и 16:00.

<h3><font color="#89DD00">Комментарий ассистента</font></h3>

Удивительно, но и здесь все верно Даниил!

## Типичное количество человек за столиком

Выясним, для какого количества человек обычно заказывают столик в разное время на примере Iskra Bar и Там Ем. Для этого напишем функцию, которая принимает данные, название ресторана, 2 времени и считает моду.

In [35]:
def table_for_func(restaurant, time_1, time_2):
    df = bo_data['table_for']
    return df[(bo_data['ordered'] >= time_1) & (bo_data['ordered'] < time_2) & (bo_data['restaurant'] == restaurant)].mode()

Проверим функцию на двух ресторанах, а также дневном и вечернем промежутке времени.

In [36]:
table_for_func('Iskra Bar','12:00','16:00')

0    2
dtype: int64

In [37]:
table_for_func('Iskra Bar','16:00','22:00')

0    1
dtype: int64

In [38]:
table_for_func('Там Ем','12:00','16:00')

0    2
dtype: int64

In [39]:
table_for_func('Там Ем','16:00','22:00')

0    1
dtype: int64

### Выводы

* Проверив типичное количество человек за столиком в Iskra Bar и "Там Ем", мы обнаружили, что в дневное время чаще всего заказывают столик на двоих, в вечернее же время чаще заказывают столик на одного.

## Общие выводы

Мы проанализировали все данные и можем сделать вывод о том, что:
- больше всего приходится заказов на такие рестораны как Iskra Bar и "Там Ем", максимальная нагрузка приходится на будние дни, при этом в выходные количество заказов не большое.
- чаще всего приложением пользуются в обеденное время, нежели в вечернее.
- исходя из данных о двух самых популярных ресторанов, столики чаще заказывают на одного или двух человек, а именно, в обеденное время - 2 человека, в вечернее - 1 человек.
- присутствовало небольшое (24) количество дубликатов данных, что может свидетельствовать о сбое при выгрузке данных, просим принять это во внимание.

<h3><font color="#89DD00">Комментарий ассистента</font></h3>

У меня не возникло критических замечаний ни к коду, ни к выводам. Все отлично, проект принят!